In [1]:
import time
import re
import codecs
import urllib2 as ul
import xml.etree.ElementTree as ET

def ReadSent(pageno,src='../han/'):
    dat=codecs.open(src+str(pageno)+'.d',encoding='utf-8')
    dat = dat.read()
    dat = Clean(dat)
    dat = dat.replace('?','.')
    dat = dat.replace('!','.')
    return [' '.join(i.split()) for i in dat.split('.')]
    
def Clean(sentence,comma=0):
    if comma==0:
        for i in ['\n','_','-','(',')','"','\'',u'▲','...','[',']',u'■','<','>','\r',u'▶',u'◀']:
            sentence=sentence.replace(i,' ')
    else:
        for i in ['\n','_','-','(',')','"','\'',u'▲','...','[',']',u'■','<','>',',','\r']:
            sentence=sentence.replace(i,' ')
    return sentence.strip()

def LoadDB():
    sent1=[]
    f = open('../lists/han.list')
    for line in f:
        tmp=ReadSent(int(line.split('.')[0]))
        for j in tmp:
            sent1.append(j)

    sent2=[]
    f = open('../lists/dat.list')
    for line in f:
        tmp=ReadSent(int(line.split('.')[0]),src='../dat/')
        for j in tmp:
            sent2.append(j)
        
    sent3=[]
    f = open('../lists/oro.list')
    for line in f:
        tmp=ReadSent(int(line.split('.')[0]),src='../oro/')
        for j in tmp:
            sent3.append(j)

    return sent1+sent2+sent3

def ShortSent(sent):
    return sorted(sent, key = lambda x: len(x))

def SearchDB(sent,token):
    matches = [i for i in sent if i.find(token)>0]
    return sorted(matches, key=lambda x: len(x))

def SearchDBList(sent,tokens):
    matches=[]
    for token in tokens:
        matches += [i for i in sent if i.find(token)>0]
    return sorted(matches, key=lambda x: len(x))

def Simplify(sent):
    return list(set(sent))

#a la https://github.com/mouuff/Google-Translate-API/blob/master/python/GoogleTranslate.py
def translate(to_translate, to_langage="auto", langage="auto"):
    to_translate=to_translate.encode('utf-8')
    agents = {'User-Agent':"Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1;SV1; .NET CLR 1.1.4322; .NET CLR 2.0.50727; .NET CLR 3.0.04506.30)"}
    before_trans = 'class="t0">'
    link = "http://translate.google.com/m?hl=%s&sl=%s&q=%s"% (to_langage, langage, to_translate.replace(" ", "+"))
    request = ul.Request(link, headers=agents)
    page = ul.urlopen(request).read()
    result = page[page.find(before_trans)+len(before_trans):]
    result = result.split("<")[0]
    return result

def LoadWordBank(wbank,path='./xml/'):
    tree = ET.parse(path+wbank)
    root = tree.getroot()
    mywords={}
    for j,k in zip(root.iter('Word'),root.iter('English')):
        mywords[j.text]=k.text
    return mywords

def WCount(sent,write=0,n=2000,name='freq.wlist'):
    mywords={}
    for i in sent:
        i=i.replace(',','')
        for j in i.split():
            try:
                mywords[j]+=1
            except KeyError:
                mywords[j]=1
    freq = sorted(mywords.keys(), key=lambda x: mywords[x])
    freq.reverse()
    if write==1:
        dat=codecs.open('./output/'+name,'w',encoding='utf-8')
        for i in freq[:n]:
            try:
                dat.write(i)
                dat.write(', '+translate(i))
                dat.write('\n')
            except UnicodeDecodeError:
                continue
        
    return mywords, freq
        
def LoadFreqList(name,src='./output/'):
    f = open(src+name)
    keys=[]
    vals=[]
    for line in f:
        tmp = line.decode('utf-8').split(',')
        keys.append(tmp[0])
        vals.append(int(tmp[1]))
    return keys,vals

def MergeFreqList(name1,name2):
    dic1={}
    keys,vals=LoadFreqList(name1)
    keys1,vals1=LoadFreqList(name2)
    for i,j in zip(keys,vals):
        dic1[i]=j
    for i,j in zip(keys1,vals1):
        try:
            dic1[i]+=j
        except KeyError:
            dic1[i]=j
    keyt=sorted(dic1.keys(), key=lambda x: dic1[x])
    keyt.reverse()
    valt=[dic1[i] for i in keyt]
    return keyt,valt

In [2]:
sent = LoadDB()

##Memrise 1000 most common words level 1

In [10]:
# 것 a thing or an object (also possible without ᄉ)
# 하다 to do
# 있다 to exist, to be (there), to have
# 수 way, method, idea, solution (used as a part of a grammar pattern)
# 나 me (informal)
# 없다 to not exist, to be absent, to not have
# 않다 to not be, to not do (two syllables)
# 사람 person, human (commonly used)
# 우리 we,our
# 그 he / that (near the listener, far from speaker)
# 아니다 to not be, no (three syllables)
# 보다 to try doing something (v.s.-아/어/여 x)
# 저것 that thing over there (far from speaker and listener)
# 보다 to see, watch, look (at)
# 같다 to be like; equal; 

tags = [u'것',u'하다',u'있다',u'수',u'나',u'없다',u'않다',u'사람',u'우리',u'그',u'아니다',u'보다',u'같다']
tags = [' '+i+' ' for i in tags]
n,nn=25,10
for tag in tags:
    stag = SearchDB(sent,tag)
    stag=Simplify(stag)
    stag=ShortSent(stag)
    print tag, len(stag)
    for i in range(min([n,len(stag)])):print stag[i]
    print ''
    print tag, translate(tag)
    for i in range(min([nn,len(stag)])):print stag[i],translate(stag[i])
    print ''

  것  23382
그런 것 몰라
아닌 것 같다
별 것 아니다
하는 것 같다
했던 것 같다
그런 것 같다
그런 것 없다
어떨 것 같아
별 것 없어요
끝난 것 같다
어려울 것 없다
그랬을 것 같다
“그런 것 같다
운명인 것 같다
힘들 것 같아요
난 질 것 같아
쉬울 것 같은가
두려울 것 없지
좋아진 것 같다
괜찮은 것 같다
시원한 것 인정
와전된 것 같다
그런 것 입니다
“그럴 것 같다
어떤 것 같은가

 것   that
그런 것 몰라 I do not know like it
아닌 것 같다 Not seem to
별 것 아니다 Stars will not
하는 것 같다 It seems that
했던 것 같다 It seems that
그런 것 같다 It seems that
그런 것 없다 I do not think so
어떨 것 같아 How about I think
별 것 없어요 I would Stars
끝난 것 같다 It seems over

 하다  356
‘~해야 하다 의무
어찌어찌 하다 보니 이겼다
많은 고민을 하다 결심하게 됐다
긴 바둑이 될 듯 하다 고 말했다
정치를 하다 보니 시간이 잘 없죠
어떻게 하다 보니 6연승을 했습니다
경기를 하다 보면 사이클이 있습니다
같이 방송을 하다 결혼까지 가게 됐다
기사생활 하다 보면 감이라는 것이 있다
척을 여러 번 하다 보면 습관이 되고요
초읽기 하다 말고 마지막이니 빨리 둬 주세요
바둑TV에서 김성룡 9단이 해설을 하다 놀란다
” 그럼 어떻게 하다 복싱을 시작하게 된 거죠
이 동네 일을 하다 보면 느낌이라는 것이 있다
그 많은 걸 하다 보면 바둑 공부는 언제 하는가
때론 내가 별 걱정을 다하는가 싶기도 하다 웃음
프로기사를 하다 실무자를 맡아보니 느낌이 어떤지
그것은 한국바둑계의 소망이기도 하다 며 덕담했다
고민을 하다 어머니께 도와달라고 간곡히 부탁드렸다
어떻게 하다 보니 5연승까지 왔는데 어리둥절합니다
지도자는 한국 방문 내내 바둑 이야기만 하다 갔다
아무래도 회사 일을 하다 보니 집중이 쉽지 않습니다
가일수를 해야